## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns

y_df = attrition_df[['Attrition', 'Department']]
y_df.head()

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [4]:
# Create a list of at least 10 column names to use as X data
X_data = ['Education','Age','DistanceFromHome','JobSatisfaction','OverTime','StockOptionLevel','WorkLifeBalance','YearsAtCompany','YearsSinceLastPromotion','NumCompaniesWorked']

# Create X_df using your selected columns
X_df = attrition_df[X_data]

# Show the data types for X_df
X_df.dtypes


Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [5]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=1)

In [6]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
from sklearn.preprocessing import LabelEncoder

In [7]:
# Do the value count for the OverTime column
X_train.value_counts('OverTime')

OverTime
No     798
Yes    304
Name: count, dtype: int64

In [8]:
# LabelEncode the OverTime column
# Create an encoder
encoder = LabelEncoder()

# Fit the encoder
encoder.fit(X_train['OverTime'])

LabelEncoder()

In [9]:
# Transform the X_train and X_test data
X_train['OverTime'] = encoder.transform(X_train['OverTime'])
X_test['OverTime'] = encoder.transform(X_test['OverTime'])

In [10]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_train_scaled = scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [11]:
# Create a OneHotEncoder for the Department column
from sklearn.preprocessing import OneHotEncoder

# Create an encoder
dept_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
y_train_dept_enc = dept_encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_dept_trans = dept_encoder.transform(y_train[['Department']])
y_test_dept_trans = dept_encoder.transform(y_test[['Department']])

y_train_dept_trans[:]

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [12]:
# Create a OneHotEncoder for the Attrition column
attr_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
y_train_attr_enc = attr_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attr_trans = attr_encoder.transform(y_train[['Attrition']])
y_test_attr_trans = attr_encoder.transform(y_test[['Attrition']])

y_train_attr_trans[:]

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

## Create, Compile, and Train the Model

In [13]:
# Find the number of columns in the X training data
input_features = len(X_train.columns)
input_features

10

In [14]:
# Find the number of columns and rows in the X_train_scaled array
print(X_train_scaled.shape)

(1102, 10)


In [15]:
# Create the input layer
input_layer = layers.Input(shape=(10,), name='input_layer')

# Create at least two shared layers
shared1 = layers.Dense(units=64, activation='relu', name='shared1')(input_layer)
shared2 = layers.Dense(units=128, activation='relu', name='shared2')(shared1)

In [16]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = layers.Dense(32, activation='relu', name='department_hidden')(shared2)

# Create the output layer
department_output = layers.Dense(3,
                             activation='softmax',
                             name='department_output')(department_hidden)


In [17]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(32, activation='relu',name='attrition_hidden')(shared2)

# Create the output layer
attrition_output = layers.Dense(2,
                             activation='sigmoid',
                             name='attrition_output')(attrition_hidden)


In [18]:
# Create the model
model = Model(inputs=input_layer, outputs=[
    department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy',
                    'attrition_output': 'binary_crossentropy'},
              metrics={'department_output': 'accuracy',
                       'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_layer (InputLayer)    [(None, 10)]                 0         []                            
                                                                                                  
 shared1 (Dense)             (None, 64)                   704       ['input_layer[0][0]']         
                                                                                                  
 shared2 (Dense)             (None, 128)                  8320      ['shared1[0][0]']             
                                                                                                  
 department_hidden (Dense)   (None, 32)                   4128      ['shared2[0][0]']             
                                                                                              

In [19]:
# Train the model
model_fit = model.fit(
    X_train_scaled,
    {
        'department_output': y_train_dept_trans,
        'attrition_output': y_train_attr_trans
    },
    epochs=100,  # You can adjust the number of epochs based on your needs
    batch_size=26,  # You  can adjust the batch size based on your available memory
    validation_split=0.2 # You can specify the validation split if you have a separate validation set
)

Epoch 1/100
34/34 [==============================] - 7s 32ms/step - loss: 1.3525 - department_output_loss: 0.8585 - attrition_output_loss: 0.4940 - department_output_accuracy: 0.6186 - attrition_output_accuracy: 0.8502 - val_loss: 1.3047 - val_department_output_loss: 0.8313 - val_attrition_output_loss: 0.4735 - val_department_output_accuracy: 0.6335 - val_attrition_output_accuracy: 0.8281
Epoch 2/100
34/34 [==============================] - 0s 14ms/step - loss: 1.1707 - department_output_loss: 0.7708 - attrition_output_loss: 0.3999 - department_output_accuracy: 0.6640 - attrition_output_accuracy: 0.8513 - val_loss: 1.2874 - val_department_output_loss: 0.8323 - val_attrition_output_loss: 0.4551 - val_department_output_accuracy: 0.6335 - val_attrition_output_accuracy: 0.8281
Epoch 3/100
34/34 [==============================] - 1s 18ms/step - loss: 1.1303 - department_output_loss: 0.7602 - attrition_output_loss: 0.3701 - department_output_accuracy: 0.6640 - attrition_output_accuracy: 0.85

In [20]:
# Evaluate the model with the testing data
results = model.evaluate(np.array(X_test_scaled), {
        'department_output': y_test_dept_trans,
        'attrition_output': y_test_attr_trans
    })

# Print the evaluation results
results

12/12 [==============================] - 0s 3ms/step - loss: 4.5744 - department_output_loss: 2.9675 - attrition_output_loss: 1.6068 - department_output_accuracy: 0.5462 - attrition_output_accuracy: 0.8071


[4.574354648590088,
 2.967520236968994,
 1.6068344116210938,
 0.5461956262588501,
 0.8070651888847351]

In [21]:
# Print the accuracy for both department and attrition
pred_categories = ['Department', 'Attrition']
for i, cat in enumerate(pred_categories):
    print(f"{cat} predictions accuracy: {results[i+3]}")

Department predictions accuracy: 0.5461956262588501
Attrition predictions accuracy: 0.8070651888847351


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1.  That is hard to say.  The accuracy was a good metric to use for the training of the model, but it only got 55% on the Department accuracy and 81% on the Attrition accuracy when used on the test data.  Maybe another metric could be used for training and testing; but, more likely, it needs a lot more data.

2.  The activation function I chose were Softmax on the Department output layer and Sigmoid on the Attrition.  Softmax was chosen since it is used for categories of 3 or more which the Department had several.  Sigmoid was used on the Attrition since it only had two possible values which makes the data binary and that is what the Sigmoid function is for.

3. One way is to add a lot more data.  Accuracy might be improved by adding some more shared layers.  However, the best way is probably to add more columns to the X_df dataframe for the original .csv file.